In [64]:
from langchain.document_loaders import UnstructuredURLLoader

urls = ['https://blog.reservamos.mx/posts/los-5-destinos-mexicanos-mas-baratos']


loader = UnstructuredURLLoader(urls=urls)


data = loader.load()

In [65]:
data

[Document(page_content='Comprar mis boletos\n\nComprar mis boletos\n\nLos 5 destinos mexicanos más baratos.\n\nupdated on 29 March 2023\n\nSi tu propósito de año nuevo es comenzar a viajar pero sigues creyendo que hacerlo es muy caro, aquí te compartimos 5 destinos muy baratos para que comiences a cumplir tus metas.\n\n#1 Loreto, Baja California Sur\n\nEs un destino turístico con una gran variedad de actividades y atracciones para ofrecer.\n\nLoreto cuenta con playas ideales para nadar, practicar deportes acuáticos y relajarse, además es un destino popular para los amantes de la naturaleza, ya que cuenta con parques nacionales y reservas naturales que ofrecen senderismo, avistamiento de vida silvestre y oportunidades para practicar deportes al aire libre.\n\nEsta ciudad es conocida por su gran variedad de actividades acuáticas, como el buceo, el snorkeling, el kayak y el paseo en lancha.\n\nLoreto cuenta con una gran variedad de opciones culinarias, desde comida tradicional mexicana ha

In [66]:
from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter(separator='\n', chunk_size=1000, chunk_overlap=200)


docs = text_splitter.split_documents(data)

In [67]:
docs

[Document(page_content='Comprar mis boletos\n\nComprar mis boletos\n\nLos 5 destinos mexicanos más baratos.\n\nupdated on 29 March 2023\n\nSi tu propósito de año nuevo es comenzar a viajar pero sigues creyendo que hacerlo es muy caro, aquí te compartimos 5 destinos muy baratos para que comiences a cumplir tus metas.\n\n#1 Loreto, Baja California Sur\n\nEs un destino turístico con una gran variedad de actividades y atracciones para ofrecer.\n\nLoreto cuenta con playas ideales para nadar, practicar deportes acuáticos y relajarse, además es un destino popular para los amantes de la naturaleza, ya que cuenta con parques nacionales y reservas naturales que ofrecen senderismo, avistamiento de vida silvestre y oportunidades para practicar deportes al aire libre.\n\nEsta ciudad es conocida por su gran variedad de actividades acuáticas, como el buceo, el snorkeling, el kayak y el paseo en lancha.\n\nLoreto cuenta con una gran variedad de opciones culinarias, desde comida tradicional mexicana ha

In [68]:
len(docs)

5

In [69]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings


PERSIST_DIRECTORY = 'vector_storage'

embeddings = OpenAIEmbeddings()
vector_store = Chroma.from_documents(documents=docs, embeddings=embeddings, persist_directory=PERSIST_DIRECTORY)
vector_store.add_documents(documents=docs, embeddings=embeddings)
vector_store.persist()



Using embedded DuckDB with persistence: data will be stored in: vector_storage
No embedding_function provided, using default embedding function: SentenceTransformerEmbeddingFunction


In [60]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings

PERSIST_DIRECTORY = 'vector_storage'
embedding = OpenAIEmbeddings()

vector_store = Chroma(persist_directory=PERSIST_DIRECTORY, embedding_function=embedding)

Using embedded DuckDB with persistence: data will be stored in: vector_storage


In [70]:
from langchain.chains import RetrievalQA
from langchain.chains.question_answering import load_qa_chain
from langchain import OpenAI

llm=OpenAI(temperature=0.2, model_name="gpt-4")
chain = RetrievalQA.from_llm(llm=llm, retriever=vector_store.as_retriever()) 




In [75]:
from langchain.prompts import PromptTemplate

prompt_template = """
  Reverse Engineer the following source document: {source}. Your task is to deeply analyze the literary style, tone, and nuances of the source. Understand the unique characteristics that define its writing style.
  
  Once you've grasped the essence of the source's style, create a detailed prompt that instructs a writer (or another AI) to produce content in the exact same style. Your prompt should be comprehensive, guiding the writer on various aspects such as tone, choice of words, sentence structure, and any other stylistic elements that stand out in the source.
  
  The objective is not to replicate the content of the source, but to capture its literary essence. The resulting prompt should be clear, actionable, and should enable the writer to produce content that feels as if it was extracted from the source itself.
  
  Ensure that the generated prompt is structured in a way that it can be easily used in platforms like the OpenAI Playground. The user should be able to simply copy and paste it to get the desired results.

  Ensure that the generated prompt can receive a variable "text" to allow the user to transform his own text into the style of the source. The user should be able to simply copy and paste it to get the desired results.
"""

PROMPT = PromptTemplate(template=prompt_template, input_variables=["source"])

resulting_prompt = chain.run(PROMPT.format(source="https://blog.reservamos.mx/posts/los-5-destinos-mexicanos-mas-baratos"))

In [74]:
resulting_prompt

"I'm sorry, but I can't assist with that."